In [0]:
from torchvision import transforms

In [0]:
def get_train_transforms(cfg: dict, ratio: float | None = None):
    if ratio is None:
        return transforms.Compose([transforms.ToTensor()])

    if ratio < 0 or ratio > 1:
        raise ValueError(f"Ratio must be between 0 and 1. Received: {ratio}")

    base_transform = transforms.Compose(
        [
            transforms.HorizontalFlip(p=cfg["flip_p"]),
            transforms.RandomRotate90(degrees=90),
            transforms.RandomBrightnessContrast(
                brightness=cfg["brightness_p"]
            ),  # Endringer i solforhold, årstid, skygge eller skydetthet
            transforms.ToTensor(),
        ]
    )
    return base_transform


def get_val_transforms():
    return transforms.Compose([transforms.ToTensor()])